In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13036,2021-11-05T17:00:00,ITA,19,Sicilia,38.115697,13.362357,311,39,350,7289,...,DECEDUTI: N. 3 IL 04/11/21 - N. 3 IL 03/11/21 ...,3.0,NaN,NaN,310145.0,0.0,3382057.0,3435160.0,ITG,ITG1
13037,2021-11-05T17:00:00,ITA,9,Toscana,43.769231,11.255889,271,27,298,5947,...,NaN,3.0,NaN,NaN,286073.0,5014.0,4822377.0,2430886.0,ITI,ITI1
13038,2021-11-05T17:00:00,ITA,10,Umbria,43.106758,12.388247,42,7,49,1270,...,NaN,0.0,NaN,NaN,65562.0,0.0,1194877.0,1028743.0,ITI,ITI2
13039,2021-11-05T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,3,0,3,64,...,NaN,0.0,NaN,NaN,11457.0,871.0,113175.0,124644.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13036,2021-11-05T17:00:00,19,Sicilia,311,39,350,7289,7639,172,466,...,310145,6817217,2580723.0,3.0,310145.0,0.0,3382057.0,3435160.0,ITG,ITG1
13037,2021-11-05T17:00:00,9,Toscana,271,27,298,5947,6245,160,381,...,291087,7253263,3121442.0,3.0,286073.0,5014.0,4822377.0,2430886.0,ITI,ITI1
13038,2021-11-05T17:00:00,10,Umbria,42,7,49,1270,1319,78,114,...,65562,2223620,458974.0,0.0,65562.0,0.0,1194877.0,1028743.0,ITI,ITI2
13039,2021-11-05T17:00:00,2,Valle d'Aosta,3,0,3,64,67,-8,8,...,12328,237819,93108.0,0.0,11457.0,871.0,113175.0,124644.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-11-05', '2021-11-04')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-11-05T17:00:00,1,Piemonte,206,21,227,4329,4556,118,386,...,386238,8145237,2622549.0,2.0,362161.0,24077.0,3673616.0,4471621.0,ITC,ITC1
1,2021-11-05T17:00:00,2,Valle d'Aosta,3,0,3,64,67,-8,8,...,12328,237819,93108.0,0.0,11457.0,871.0,113175.0,124644.0,ITC,ITC2
2,2021-11-05T17:00:00,3,Lombardia,336,48,384,10172,10556,303,840,...,897249,17880354,5857032.0,3.0,828846.0,68403.0,11373132.0,6507222.0,ITC,ITC4
3,2021-11-05T17:00:00,5,Veneto,191,41,232,11601,11833,417,792,...,483465,14269586,2200193.0,5.0,467915.0,15550.0,7161775.0,7107811.0,ITH,ITH3
4,2021-11-05T17:00:00,6,Friuli Venezia Giulia,108,16,124,2675,2799,227,410,...,118502,3294737,865175.0,2.0,102794.0,15708.0,2265301.0,1029436.0,ITH,ITH4
5,2021-11-05T17:00:00,7,Liguria,80,11,91,1260,1351,28,118,...,115425,2691612,921816.0,1.0,115425.0,0.0,1727734.0,963878.0,ITC,ITC3
6,2021-11-05T17:00:00,8,Emilia-Romagna,302,40,342,7507,7849,283,618,...,434177,9767926,2167793.0,4.0,433788.0,389.0,6221491.0,3546435.0,ITH,ITH5
7,2021-11-05T17:00:00,9,Toscana,271,27,298,5947,6245,160,381,...,291087,7253263,3121442.0,3.0,286073.0,5014.0,4822377.0,2430886.0,ITI,ITI1
8,2021-11-05T17:00:00,10,Umbria,42,7,49,1270,1319,78,114,...,65562,2223620,458974.0,0.0,65562.0,0.0,1194877.0,1028743.0,ITI,ITI2
9,2021-11-05T17:00:00,11,Marche,61,21,82,2489,2571,129,252,...,116870,1644763,953361.0,0.0,116870.0,0.0,1411991.0,232772.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-11-04T17:00:00,1,Piemonte,206,21,227,4211,4438,62,286,...,385852,8088886,2617690.0,3.0,361923.0,23929.0,3666803.0,4422083.0,ITC,ITC1
1,2021-11-04T17:00:00,2,Valle d'Aosta,3,0,3,72,75,-7,0,...,12320,235900,92994.0,0.0,11453.0,867.0,113061.0,122839.0,ITC,ITC2
2,2021-11-04T17:00:00,3,Lombardia,330,49,379,9874,10253,211,745,...,896409,17766028,5847440.0,3.0,828227.0,68182.0,11350266.0,6415762.0,ITC,ITC4
3,2021-11-04T17:00:00,5,Veneto,184,39,223,11193,11416,242,734,...,482673,14168592,2196958.0,7.0,467176.0,15497.0,7145021.0,7023571.0,ITH,ITH3
4,2021-11-04T17:00:00,6,Friuli Venezia Giulia,98,16,114,2458,2572,263,483,...,118094,3268935,863649.0,0.0,102416.0,15678.0,2258625.0,1010310.0,ITH,ITH4
5,2021-11-04T17:00:00,7,Liguria,78,10,88,1235,1323,91,91,...,115307,2676839,920323.0,0.0,115307.0,0.0,1724572.0,952267.0,ITC,ITC3
6,2021-11-04T17:00:00,8,Emilia-Romagna,297,38,335,7231,7566,68,328,...,433560,9739470,2165063.0,8.0,433173.0,387.0,6207368.0,3532102.0,ITH,ITH5
7,2021-11-04T17:00:00,9,Toscana,274,26,300,5785,6085,158,385,...,290706,7222314,3112429.0,3.0,285705.0,5001.0,4814418.0,2407896.0,ITI,ITI1
8,2021-11-04T17:00:00,10,Umbria,39,7,46,1195,1241,29,99,...,65448,2213149,458397.0,0.0,65448.0,0.0,1192648.0,1020501.0,ITI,ITI2
9,2021-11-04T17:00:00,11,Marche,59,22,81,2361,2442,100,159,...,116618,1641258,951563.0,2.0,116618.0,0.0,1409314.0,231944.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-11-05T17:00:00,1,Piemonte,206,21,227,4329,4556,118,386,...,8145237,2622549.0,2.0,362161.0,24077.0,3673616.0,4471621.0,ITC,ITC1,4.341375e+06
1,2021-11-05T17:00:00,2,Valle d'Aosta,3,0,3,64,67,-8,8,...,237819,93108.0,0.0,11457.0,871.0,113175.0,124644.0,ITC,ITC2,1.255010e+05
2,2021-11-05T17:00:00,3,Lombardia,336,48,384,10172,10556,303,840,...,17880354,5857032.0,3.0,828846.0,68403.0,11373132.0,6507222.0,ITC,ITC4,1.010397e+07
3,2021-11-05T17:00:00,5,Veneto,191,41,232,11601,11833,417,792,...,14269586,2200193.0,5.0,467915.0,15550.0,7161775.0,7107811.0,ITH,ITH3,4.907704e+06
4,2021-11-05T17:00:00,6,Friuli Venezia Giulia,108,16,124,2675,2799,227,410,...,3294737,865175.0,2.0,102794.0,15708.0,2265301.0,1029436.0,ITH,ITH4,1.211357e+06
5,2021-11-05T17:00:00,7,Liguria,80,11,91,1260,1351,28,118,...,2691612,921816.0,1.0,115425.0,0.0,1727734.0,963878.0,ITC,ITC3,1.543127e+06
6,2021-11-05T17:00:00,8,Emilia-Romagna,302,40,342,7507,7849,283,618,...,9767926,2167793.0,4.0,433788.0,389.0,6221491.0,3546435.0,ITH,ITH5,4.467118e+06
7,2021-11-05T17:00:00,9,Toscana,271,27,298,5947,6245,160,381,...,7253263,3121442.0,3.0,286073.0,5014.0,4822377.0,2430886.0,ITI,ITI1,3.722729e+06
8,2021-11-05T17:00:00,10,Umbria,42,7,49,1270,1319,78,114,...,2223620,458974.0,0.0,65562.0,0.0,1194877.0,1028743.0,ITI,ITI2,8.802850e+05
9,2021-11-05T17:00:00,11,Marche,61,21,82,2489,2571,129,252,...,1644763,953361.0,0.0,116870.0,0.0,1411991.0,232772.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-11-05T17:00:00,1,Piemonte,206,21,227,4329,4556,118,386,...,4471621.0,ITC,ITC1,4.341375e+06,Piemonte,206,5824,21,628,99
1,2021-11-05T17:00:00,2,Valle d'Aosta,3,0,3,64,67,-8,8,...,124644.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,3,83,0,33,0
2,2021-11-05T17:00:00,3,Lombardia,336,48,384,10172,10556,303,840,...,6507222.0,ITC,ITC4,1.010397e+07,Lombardia,336,6369,48,1530,0
3,2021-11-05T17:00:00,5,Veneto,191,41,232,11601,11833,417,792,...,7107811.0,ITH,ITH3,4.907704e+06,Veneto,191,6000,41,1000,0
4,2021-11-05T17:00:00,6,Friuli Venezia Giulia,108,16,124,2675,2799,227,410,...,1029436.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,108,1277,16,175,0
5,2021-11-05T17:00:00,7,Liguria,80,11,91,1260,1351,28,118,...,963878.0,ITC,ITC3,1.543127e+06,Liguria,80,1717,11,217,12
6,2021-11-05T17:00:00,8,Emilia-Romagna,302,40,342,7507,7849,283,618,...,3546435.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,302,7920,40,889,0
7,2021-11-05T17:00:00,9,Toscana,271,27,298,5947,6245,160,381,...,2430886.0,ITI,ITI1,3.722729e+06,Toscana,271,5033,27,570,36
8,2021-11-05T17:00:00,10,Umbria,42,7,49,1270,1319,78,114,...,1028743.0,ITI,ITI2,8.802850e+05,Umbria,42,662,7,84,43
9,2021-11-05T17:00:00,11,Marche,61,21,82,2489,2571,129,252,...,232772.0,ITI,ITI3,1.518400e+06,Marche,61,966,21,214,62


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-11-05T17:00:00,3,Lombardia,336,48,384,10172,10556,303,840,...,3.14,298,95,114326,9592.0,7,530,0.73,0.00831,8.88
1,2021-11-05T17:00:00,5,Veneto,191,41,232,11601,11833,417,792,...,4.10,408,58,100994,3235.0,1,374,0.78,0.01614,9.85
2,2021-11-05T17:00:00,15,Campania,262,18,280,9663,9943,343,722,...,3.24,343,107,23282,2377.0,7,372,3.10,0.01248,8.12
3,2021-11-05T17:00:00,12,Lazio,474,60,534,9715,10249,293,716,...,6.36,264,52,39060,4695.0,7,416,1.83,0.01221,6.78
4,2021-11-05T17:00:00,8,Emilia-Romagna,302,40,342,7507,7849,283,618,...,4.50,276,290,28456,2730.0,4,330,2.17,0.01383,9.72
5,2021-11-05T17:00:00,19,Sicilia,311,39,350,7289,7639,172,466,...,4.46,161,94,24601,5162.0,9,285,1.89,0.00938,6.24
6,2021-11-05T17:00:00,6,Friuli Venezia Giulia,108,16,124,2675,2799,227,410,...,9.14,217,-73,25802,1526.0,6,175,1.59,0.03385,9.78
7,2021-11-05T17:00:00,1,Piemonte,206,21,227,4329,4556,118,386,...,3.34,118,100,56351,4859.0,3,265,0.68,0.00889,8.90
8,2021-11-05T17:00:00,9,Toscana,271,27,298,5947,6245,160,381,...,4.74,162,-4,30949,9013.0,3,218,1.23,0.01023,7.82
9,2021-11-05T17:00:00,11,Marche,61,21,82,2489,2571,129,252,...,9.81,128,93,3505,1798.0,2,121,7.19,0.01660,7.70


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Marche,21,193,214,9.81
1,Friuli Venezia Giulia,16,159,175,9.14
2,Umbria,7,77,84,8.33
3,Lazio,60,883,943,6.36
4,Liguria,11,206,217,5.07
5,Toscana,27,543,570,4.74
6,Calabria,8,166,174,4.60
7,Emilia-Romagna,40,849,889,4.50
8,Sicilia,39,835,874,4.46
9,Puglia,20,462,482,4.15


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))